In [0]:
%pip install git+https://github.com/databricks-industry-solutions/x12-edi-parser@raven-eda-hl-segments

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Cloning https://github.com/databricks-industry-solutions/x12-edi-parser (to revision raven-eda-hl-segments) to /tmp/pip-req-build-fp797b52
  Running command git clone --filter=blob:none --quiet https://github.com/databricks-industry-solutions/x12-edi-parser /tmp/pip-req-build-fp797b52
  Running command git checkout -b raven-eda-hl-segments --track origin/raven-eda-hl-segments
  Switched to a new branch 'raven-eda-hl-segments'
  branch 'raven-eda-hl-segments' set up to track 'origin/raven-eda-hl-segments'.
  Resolved https://github.com/databricks-industry-solutions/x12-edi-parser to commit f1cea0934f28bcce22d153772d8de9a11c189052
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

Is the EDI data 837I or 837P transactions or neither?

In [0]:
from databricksx12.format import *
from databricksx12.edi import *
x =  EDIManager(EDI(open("/Volumes/raven_catalog/x12_edi_parser/sampledata/CHPW_Claimdata.txt", "rb").read().decode("utf-8")))

import json
print(json.dumps(x.flatten(x.data), indent=4))

{
    "EDI.sender_tax_id": "ZZ",
    "list": [
        {
            "FunctionalGroup.receiver": "123456789",
            "FunctionalGroup.sender": "CLEARINGHOUSE",
            "FunctionalGroup.transaction_datetime": "20180508:0833",
            "FunctionalGroup.transaction_type": "222",
            "list": [
                {
                    "Transaction.transaction_type": "222"
                },
                {
                    "Transaction.transaction_type": "222"
                },
                {
                    "Transaction.transaction_type": "222"
                },
                {
                    "Transaction.transaction_type": "222"
                },
                {
                    "Transaction.transaction_type": "222"
                }
            ]
        }
    ]
}


In [0]:
from databricksx12 import *
from databricksx12.hls import *
import json
from pyspark.sql.functions import input_file_name

# Initialize HealthcareManager
hm = HealthcareManager()
df = spark.read.text("/Volumes/raven_catalog/x12_edi_parser/sampledata/CHPW_Claimdata.txt", wholetext = True)

# Process the DataFrame and convert to RDD
rdd = (
 df.withColumn("filename", input_file_name()).rdd
  .map(lambda x: (x.asDict().get("filename"),x.asDict().get("value")))
  .map(lambda x: (x[0], EDI(x[1])))
  .map(lambda x: { **{'filename': x[0]}, **hm.to_json(x[1])} )
  .map(lambda x: json.dumps(x))
)

# Convert RDD to DataFrame
edi_data_df = spark.read.json(rdd)

In [0]:
display(edi_data_df)

EDI.sender_tax_id FuncitonalGroup filename ZZ List(List(123456789, CLEARINGHOUSE, 20180508:0833, 222, List(List(List(List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 20, 1, 7349065509, Rendering Provider, 57, PROVIDER JAMES , 1112223338, 1805080AV3648339, List(List(List(Rendering Provider, PROVIDER JAMES , 1112223338, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, WA, 987 65TH PL, Individual, 986640001)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER JAMES , 1112223338, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001))), 222), List(List(List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 50.1, 1, 7349065730, Rendering Provider, 57, PROVIDER SUSAN , 1112223346, 1805080AV3648347, List(List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, WA, 765 43RD ST, Individual, 986640002)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, Self, WA, 765 43RD ST, Individual, 986640002), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, Self, WA, 765 43RD ST, Individual, 986640002))), 222), List(List(List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 11.64, 1, 7349065492, Rendering Provider, 57, PROVIDER SUSAN , 1112223346, 1805080AV3648340, List(List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, WA, 987 65TH PL, Individual, 986640001)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001))), 222), List(List(List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 234, 1, 7349064290, Rendering Provider, 53, PROVIDE

In [0]:
from pyspark.sql.functions import explode, col
from pyspark.sql.types import StructType, ArrayType

# Extract FunctionalGroups and Transactions
functional_groups_df = edi_data_df.select(explode(col("FuncitonalGroup")).alias("FunctionalGroup"))

display(functional_groups_df)

FunctionalGroup


In [0]:
functional_groups_df.printSchema()

root
 |-- FunctionalGroup: struct (nullable = true)
 |    |-- FunctionalGroup.receiver: string (nullable = true)
 |    |-- FunctionalGroup.sender: string (nullable = true)
 |    |-- FunctionalGroup.transaction_datetime: string (nullable = true)
 |    |-- FunctionalGroup.transaction_type: string (nullable = true)
 |    |-- Transactions: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- Claims: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- billing_provider: struct (nullable = true)
 |    |    |    |    |    |    |-- city: string (nullable = true)
 |    |    |    |    |    |    |-- entity_type: string (nullable = true)
 |    |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |    |    |-- npi: string (nullable = true)
 |    |    |    |    |    |    |-- state: string (nullable = true)
 |    |    |    |    |    |    |-- street: string (nullable 

In [0]:
functional_groups_flat = functional_groups_df.select(
    col("FunctionalGroup.`FunctionalGroup.receiver`").alias("receiver"),
    col("FunctionalGroup.`FunctionalGroup.sender`").alias("sender"),
    col("FunctionalGroup.`FunctionalGroup.transaction_datetime`").alias("transaction_datetime"),
    col("FunctionalGroup.`FunctionalGroup.transaction_type`").alias("transaction_type"),
    explode(col("FunctionalGroup.Transactions")).alias("Transaction")
)

display(functional_groups_flat)

receiver,sender,transaction_datetime,transaction_type,Transaction
123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 20, 1, 7349065509, Rendering Provider, 57, PROVIDER JAMES , 1112223338, 1805080AV3648339, List(List(List(Rendering Provider, PROVIDER JAMES , 1112223338, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, WA, 987 65TH PL, Individual, 986640001)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER JAMES , 1112223338, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001))), 222)"
123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 50.1, 1, 7349065730, Rendering Provider, 57, PROVIDER SUSAN , 1112223346, 1805080AV3648347, List(List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, WA, 765 43RD ST, Individual, 986640002)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, Self, WA, 765 43RD ST, Individual, 986640002), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, Self, WA, 765 43RD ST, Individual, 986640002))), 222)"
123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 11.64, 1, 7349065492, Rendering Provider, 57, PROVIDER SUSAN , 1112223346, 1805080AV3648340, List(List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, WA, 987 65TH PL, Individual, 986640001)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001))), 222)"
123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCO

In [0]:
transactions_flat = functional_groups_flat.select(
    col("receiver"),
    col("sender"),
    col("transaction_datetime"),
    col("transaction_type"),
    explode(col("Transaction.Claims")).alias("Claim")
)

display(transactions_flat)

receiver,sender,transaction_datetime,transaction_type,Claim
123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 20, 1, 7349065509, Rendering Provider, 57, PROVIDER JAMES , 1112223338, 1805080AV3648339, List(List(List(Rendering Provider, PROVIDER JAMES , 1112223338, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, WA, 987 65TH PL, Individual, 986640001)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER JAMES , 1112223338, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001))"
123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 50.1, 1, 7349065730, Rendering Provider, 57, PROVIDER SUSAN , 1112223346, 1805080AV3648347, List(List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, WA, 765 43RD ST, Individual, 986640002)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, Self, WA, 765 43RD ST, Individual, 986640002), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 765123, PATIENT SUSAN E, Self, WA, 765 43RD ST, Individual, 986640002))"
123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes, VANCOUVER, 11.64, 1, 7349065492, Rendering Provider, 57, PROVIDER SUSAN , 1112223346, 1805080AV3648340, List(List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization))), WA, 12345 MAIN ST SUITE A1, Individual, 98662), TODO, List(List(List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, WA, 987 65TH PL, Individual, 986640001)), List(List(Payer, CHPWA, COMMUNITY HEALTH PLAN OF WASHINGTON, Organization)), List(List(Rendering Provider, PROVIDER SUSAN , 1112223346, Individual)), List(List(VANCOUVER, Service Facility, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST SUITE A1, Organization, 98662))), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001), List(Unknown, CHPWA, 123456789, Organization), List(CLEARINGHOUSE CLIENT SERVICES, List(List(List(Telephone, Fax, 8005551212, 8005551212))), Unknown, 987654321, CLEARINGHOUSE LLC, Organization), List(VANCOUVER, Insured, 987321, SUBSCRIBER JOHN J, Self, WA, 987 65TH PL, Individual, 986640001))"
123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(VANCOUVER, Billing Provider, BH CLINIC OF VANCOUVER, 1122334455, WA, 12345 MAIN ST, Organization, 98662), List(Yes

Parse the nested Claims data

In [0]:

def extract_fields(schema, prefix=""):
  def get_field_name(field, prefix):
      return f"{prefix}.{field.name}" if prefix else field.name

  return sum(
    [
      (
        extract_fields(field.dataType, get_field_name(field, prefix))
        if isinstance(field.dataType, StructType) else
        (
            [explode(col(get_field_name(field, prefix))).alias(get_field_name(field, prefix).replace(".", "_"))]
            + extract_fields(field.dataType.elementType, get_field_name(field, prefix).replace(".", "_"))
            if isinstance(field.dataType, ArrayType) and isinstance(field.dataType.elementType, StructType) else
            [col(get_field_name(field, prefix)).alias(get_field_name(field, prefix).replace(".", "_"))]
        )
      )
        for field in schema.fields
    ],
    []
)

# Get the schema of the Claim struct
claim_schema = transactions_flat.select("Claim").schema[0].dataType

# Extract all fields from the Claim schema
claim_fields = extract_fields(claim_schema, "Claim")

all_fields = [
    col("receiver"),
    col("sender"),
    col("transaction_datetime"),
    col("transaction_type"),
] + claim_fields

In [0]:
# Select all fields dynamically
claims_flat = transactions_flat.select(*all_fields)


In [0]:
# Check for invalid characters in Column headers
from pyspark.sql import DataFrame

def replace_spaces_in_column_names(df: DataFrame) -> DataFrame:
    for col_name in df.columns:
        if ' ' in col_name:
            df = df.withColumnRenamed(col_name, col_name.replace(' ', '_'))
    return df

claims_flat = replace_spaces_in_column_names(claims_flat)

In [0]:
display(claims_flat)

receiver,sender,transaction_datetime,transaction_type,Claim_billing_provider_city,Claim_billing_provider_entity_type,Claim_billing_provider_name,Claim_billing_provider_npi,Claim_billing_provider_state,Claim_billing_provider_street,Claim_billing_provider_type,Claim_billing_provider_zip,Claim_claim_header_benefits_assign_flag,Claim_claim_header_city,Claim_claim_header_claim_amount,Claim_claim_header_claim_code_freq,Claim_claim_header_claim_id,Claim_claim_header_entity_type,Claim_claim_header_facility_type_code,Claim_claim_header_name,Claim_claim_header_npi,Claim_claim_header_patient_id,Claim_claim_header_providers_Rendering_Provider,Claim_claim_header_providers_Rendering_Provider_entity_type,Claim_claim_header_providers_Rendering_Provider_name,Claim_claim_header_providers_Rendering_Provider_npi,Claim_claim_header_providers_Rendering_Provider_type,Claim_claim_header_providers_Service_Facility,Claim_claim_header_providers_Service_Facility_entity_type,Claim_claim_header_providers_Service_Facility_name,Claim_claim_header_providers_Service_Facility_npi,Claim_claim_header_providers_Service_Facility_type,Claim_claim_header_state,Claim_claim_header_street,Claim_claim_header_type,Claim_claim_header_zip,Claim_claim_lines,Claim_grouped_subscriber_entities_Insured,Claim_grouped_subscriber_entities_Insured_city,Claim_grouped_subscriber_entities_Insured_entity_type,Claim_grouped_subscriber_entities_Insured_id,Claim_grouped_subscriber_entities_Insured_name,Claim_grouped_subscriber_entities_Insured_state,Claim_grouped_subscriber_entities_Insured_street,Claim_grouped_subscriber_entities_Insured_type,Claim_grouped_subscriber_entities_Insured_zip,Claim_grouped_subscriber_entities_Payer,Claim_grouped_subscriber_entities_Payer_entity_type,Claim_grouped_subscriber_entities_Payer_id,Claim_grouped_subscriber_entities_Payer_name,Claim_grouped_subscriber_entities_Payer_type,Claim_grouped_subscriber_entities_Rendering_Provider,Claim_grouped_subscriber_entities_Rendering_Provider_entity_type,Claim_grouped_subscriber_entities_Rendering_Provider_name,Claim_grouped_subscriber_entities_Rendering_Provider_npi,Claim_grouped_subscriber_entities_Rendering_Provider_type,Claim_grouped_subscriber_entities_Service_Facility,Claim_grouped_subscriber_entities_Service_Facility_city,Claim_grouped_subscriber_entities_Service_Facility_entity_type,Claim_grouped_subscriber_entities_Service_Facility_name,Claim_grouped_subscriber_entities_Service_Facility_npi,Claim_grouped_subscriber_entities_Service_Facility_state,Claim_grouped_subscriber_entities_Service_Facility_street,Claim_grouped_subscriber_entities_Service_Facility_type,Claim_grouped_subscriber_entities_Service_Facility_zip,Claim_patient_city,Claim_patient_entity_type,Claim_patient_id,Claim_patient_name,Claim_patient_relationship_to_insured,Claim_patient_state,Claim_patient_street,Claim_patient_type,Claim_patient_zip,Claim_reciever_entity_type,Claim_reciever_id,Claim_reciever_name,Claim_reciever_type,Claim_submitter_contact_name,Claim_submitter_contacts_primary,Claim_submitter_contacts_primary_contact_method,Claim_submitter_contacts_primary_contact_method_2,Claim_submitter_contacts_primary_contact_number,Claim_submitter_contacts_primary_contact_number_2,Claim_submitter_entity_type,Claim_submitter_id,Claim_submitter_name,Claim_submitter_type,Claim_subscriber_city,Claim_subscriber_entity_type,Claim_subscriber_id,Claim_subscriber_name,Claim_subscriber_relationship_to_insured,Claim_subscriber_state,Claim_subscriber_street,Claim_subscriber_type,Claim_subscriber_zip
123456789,CLEARINGHOUSE,20180508:0833,222,VANCOUVER,Billing Provider,BH CLINIC OF VANCOUVER,1122334455,WA,12345 MAIN ST,Organization,98662,Yes,VANCOUVER,20,1,7349065509,Rendering Provider,57,PROVIDER JAMES,1112223338,1805080AV3648339,"List(Rendering Provider, PROVIDER JAMES , 1112223338, Individual)",Rendering Provider,PROVIDER JAMES,1112223338,Individual,"List(Service Facility, BH CLINIC OF VANCOUVER, 1122334455, Organization)",Service Facility,BH CLINIC OF VANCOUV

### Write table to Unity Catalog

In [0]:
# Define the catalog and schema names
catalog_name = "raven_catalog"
schema_name = "x12_edi_parser"

# Define the path where you want to save the claims_flat DataFrame as a UC table
uc_table_name = f"{catalog_name}.{schema_name}.claims_flat_table"

# Save the claims_flat DataFrame as a Delta table in your Unity catalog
claims_flat.write.format("delta").mode("overwrite").saveAsTable(uc_table_name)

### The overall JSON data outside of Spark

In [0]:
from databricksx12 import EDI
from databricksx12.hls.healthcare import HealthcareManager

hm = HealthcareManager()
edi =  EDI(open("/Volumes/raven_catalog/x12_edi_parser/sampledata/CHPW_Claimdata.txt", "rb").read().decode("utf-8"))

#Returns parsed claim data
hm.from_edi(edi) 

#Print in json format
print(json.dumps(hm.to_json(edi), indent=4)) 

{
    "EDI.sender_tax_id": "ZZ",
    "FuncitonalGroup": [
        {
            "FunctionalGroup.receiver": "123456789",
            "FunctionalGroup.sender": "CLEARINGHOUSE",
            "FunctionalGroup.transaction_datetime": "20180508:0833",
            "FunctionalGroup.transaction_type": "222",
            "Transactions": [
                {
                    "Transaction.transaction_type": "222",
                    "Claims": [
                        {
                            "submitter": {
                                "contact_name": "CLEARINGHOUSE CLIENT SERVICES",
                                "contacts": {
                                    "primary": [
                                        {
                                            "contact_method": "Telephone",
                                            "contact_number": "8005551212",
                                            "contact_method_2": "Fax",
                                            "contact